# Pipeline with Feature Union

In [86]:
import pandas as pd
import numpy as np

In [87]:
file=r'G:/EdData/Existing Base.csv'

bd=pd.read_csv(file)

In [88]:
bd.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Revenue Grid,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,1,Zero,51-55,Partner,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,...,19.99000,0.00000,0.00000,1,Female,Wales,74.67000,18.66000,32.32000,89.43000
1,2,Zero,55-60,Single/Never Married,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,...,0.00000,0.00000,0.00000,2,Female,North West,20.19000,0.00000,4.33000,22.78000
2,3,Zero,26-30,Single/Never Married,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,...,0.00000,3.49000,0.00000,2,Male,North,98.06000,31.07000,80.96000,171.78000
3,5,Zero,18-21,Single/Never Married,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,...,0.00000,0.00000,0.00000,2,Female,West Midlands,4.10000,14.15000,17.57000,-41.70000
4,6,Zero,45-50,Partner,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,...,0.00000,45.91000,25.98000,2,Female,Scotland,70.16000,55.86000,80.44000,235.02000


In [89]:
bd.nunique()

REF_NO                             10155
children                               5
age_band                              13
status                                 5
occupation                             9
occupation_partner                     9
home_status                            5
family_income                         13
self_employed                          2
self_employed_partner                  2
year_last_moved                       95
TVarea                                14
post_code                          10040
post_area                           2039
Average Credit Card Transaction     1411
Balance Transfer                    2183
Term Deposit                        1419
Life Insurance                      3111
Medical Insurance                   1589
Average A/C Balance                 2223
Personal Loan                       1760
Investment in Mutual Fund           2470
Investment Tax Saving Bond           832
Home Loan                            884
Online Purchase 

In [90]:
bd.dtypes

REF_NO                               int64
children                            object
age_band                            object
status                              object
occupation                          object
occupation_partner                  object
home_status                         object
family_income                       object
self_employed                       object
self_employed_partner               object
year_last_moved                      int64
TVarea                              object
post_code                           object
post_area                           object
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan  

In [63]:
from sklearn.base import BaseEstimator, TransformerMixin

In [64]:
class VarTypeSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self,vartype,ignore_var):
        self.vartype=vartype
        self.ignore_var=ignore_var
    
    def fit(self,x,y=None):
        return self
    
    def transform(self,X):
        return X.select_dtypes(self.vartype).drop(self.ignore_var,axis=1)

In [65]:
class get_dummies_PipeLineFriendly(BaseEstimator, TransformerMixin):
    
    def __init__(self,freq_cutoff=0):
        self.freq_cutoff=freq_cutoff
        self.var_cat_dict={}
        
    def fit(self,x,y=None):
        data_cols=x.columns
        for col in data_cols:
            k=x[col].value_counts()
            cats=k.index[k>self.freq_cutoff][:-1]
            self.var_cat_dict[col]=cats
        return self
            
    def transform(self,x,y=None):
        dummy_data=x.copy()
        for col in self.var_cat_dict.keys():
            for cat in self.var_cat_dict[col]:
                name=col+'_'+cat
                dummy_data[name]=(dummy_data[col]==cat).astype(int)
            del dummy_data[col]
        return dummy_data

In [66]:
from sklearn.pipeline import Pipeline,FeatureUnion

In [91]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [92]:
bd_train,bd_test=train_test_split(bd,test_size=0.2,random_state=2)

In [93]:
x_train=bd_train.drop('Revenue Grid',axis=1)
x_test=bd_test.drop('Revenue Grid',axis=1)
y_train=bd_train['Revenue Grid']
y_test=bd_test['Revenue Grid']

In [94]:
bd.dtypes

REF_NO                               int64
children                            object
age_band                            object
status                              object
occupation                          object
occupation_partner                  object
home_status                         object
family_income                       object
self_employed                       object
self_employed_partner               object
year_last_moved                      int64
TVarea                              object
post_code                           object
post_area                           object
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan  

In [95]:
cat_pipe=Pipeline([
    ('cat_var',VarTypeSelector(['object'],ignore_var=['post_code','post_area'])),
    ('dummies',get_dummies_PipeLineFriendly(100))
])

In [96]:
pipe2=Pipeline([
    ('features',FeatureUnion([
        ('cat_pipe',cat_pipe),
        ('num_var',VarTypeSelector(['int64','float64'],ignore_var=['REF_NO']))
    ])),
    ('clf',LogisticRegression())
])

In [97]:
pipe2.fit(x_train,y_train)

C:\Users\Karthik\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('cat_pipe',
                                                 Pipeline(memory=None,
                                                          steps=[('cat_var',
                                                                  VarTypeSelector(ignore_var=['post_code',
                                                                                              'post_area'],
                                                                                  vartype=['object'])),
                                                                 ('dummies',
                                                                  get_dummies_PipeLineFriendly(freq_cutoff=100))],
                                                          verbose=False)),
                                                ('num_var',
                                                 Va

In [98]:
pipe2.predict_proba(x_test)

array([[0.00293776, 0.99706224],
       [0.00653154, 0.99346846],
       [0.0056417 , 0.9943583 ],
       ...,
       [0.39626731, 0.60373269],
       [0.09076388, 0.90923612],
       [0.01369318, 0.98630682]])

## Save python objects to use later

In [99]:
from sklearn.externals import joblib

In [100]:
joblib.dump(pipe2,'my_model_pipeline.pkl')

['my_model_pipeline.pkl']

## New Data Scoring

In [101]:
file1=r'G:/EdData/Existing Base_Scoring.csv'

new=pd.read_csv(file1)

In [102]:
mymodel=open('my_model_pipeline.pkl','rb')
pipe=joblib.load(mymodel)

In [103]:
pipe.predict_proba(new)

array([[6.91445382e-01, 3.08554618e-01],
       [1.96655252e-02, 9.80334475e-01],
       [2.23717147e-02, 9.77628285e-01],
       [1.28025292e-02, 9.87197471e-01],
       [2.45225343e-02, 9.75477466e-01],
       [4.19738561e-02, 9.58026144e-01],
       [5.61844140e-04, 9.99438156e-01],
       [2.47824369e-04, 9.99752176e-01],
       [3.29162503e-03, 9.96708375e-01],
       [7.91559364e-03, 9.92084406e-01],
       [1.35062488e-02, 9.86493751e-01],
       [1.35901894e-02, 9.86409811e-01],
       [3.69768641e-02, 9.63023136e-01],
       [2.25021830e-05, 9.99977498e-01],
       [3.34023148e-02, 9.66597685e-01],
       [7.03922740e-04, 9.99296077e-01],
       [2.97401732e-03, 9.97025983e-01],
       [7.04280126e-02, 9.29571987e-01],
       [6.90326792e-03, 9.93096732e-01],
       [3.24732346e-02, 9.67526765e-01],
       [4.63857211e-02, 9.53614279e-01],
       [2.71086008e-02, 9.72891399e-01],
       [1.70933068e-04, 9.99829067e-01],
       [6.38062766e-04, 9.99361937e-01],
       [7.136065

In [104]:
new.shape

(100, 31)